## Importação do Gurobi

In [53]:
import gurobipy as gp

## Parâmetros da Instância 1

In [54]:
# Número de vagões
n = 2           
# Número de trechos
m = 6           
# Número de atividades
o = 15          
# Número de trens
v = 2           
# Tempo de circulação de um vagão j em um trecho q
P = 5           
# Número grande o suficiente
M = 2880        
# Vagões do trem de saída
TRAIN_OUT = [1,1]  
# Vagões do trem de entrada
TRAIN_IN = [0,0]  
# AMV[q,s] = 1 se o trecho q é ligado ao trecho s (q vizinho de s ou existe AMV entre q e s); 0 caso contrário
AMV = [[1,1,0,0,0,0],
       [1,1,1,0,1,0],
       [0,1,1,1,0,0],
       [0,0,1,1,0,0],
       [0,1,0,0,1,1],
       [0,0,0,0,1,1]]   
# PCH[j] = trecho do pátio onde o vagão j estará posicionado ao receber o trem t ∈ Tα|j ∈ t
PCH = [2,1]
# PSA[j] = trecho do pátio onde o vagão j deverá estar posicionado para que o trem t ∈ Tβ |j ∈ t possa sair
PSA = [1,2]
# RELEASE[j] = instante mais cedo que o vagão j está disponível para entrar no pátio (release date), ou seja, horário de chegada do trem t ∈ Tα|j ∈ t
RELEASE = [10,10]

## Definição de Modelo MILP

In [55]:
model = gp.Model("milp")

## Variáveis de Decisão

In [56]:
# y[i,j] = instante de início da i-ésima operação do vagão j
y = model.addVars(o,n, vtype=gp.GRB.CONTINUOUS, name="y")         
# z[i,q,j] = 1 se a i-ésima operação do vagão j acontece no trecho q; 0 caso contrário
z = model.addVars(o,m,n, vtype=gp.GRB.BINARY, name="z")           
# x[q,i,j,k,l] = 1 se a i-ésima operação do vagão j precede a k-ésima operação do vagão l no trecho q; 0 caso contrário
x = model.addVars(m,o,n,o,n, vtype=gp.GRB.BINARY, name="x")       
# f[i,j] = 1 se a i-ésima operação do vagão j é a última, antes da partida do trem t ∈ Tβ |j ∈ t; 0 caso contrário
f = model.addVars(o,n, vtype=gp.GRB.BINARY, name="f")             
# c[j] = instante de conclusão do vagão j, horário de partida do trem t ∈ Tβ|j ∈ t
c = model.addVars(n, vtype=gp.GRB.CONTINUOUS, name="c")           
# c_max = horário da partida do último vagão j ∈ t, t ∈ Tβ (makespan)
c_max = model.addVar(vtype=gp.GRB.CONTINUOUS, name="c_max")       

## Função Objetivo

In [57]:
model.setObjective(c_max, gp.GRB.MINIMIZE)

## Restrições do Problema

In [58]:
# Equação 2 / (4-2)
for j in range(n):
  for i in range(o-1):
    model.addConstr(y[i+1,j] >= y[i,j] + (P * gp.quicksum(z[i,q,j] for q in range(m))))

In [59]:
# Equação 3 / Equação não se encontra no arquivo de tese
for j in range(n):
  for l in range(n):
    for i in range(o-1):
      for k in range(o):
        for q in range(m):
          model.addConstr(y[i+1,j] >= y[k,l] - (M * x[q,i,j,k,l]) - (M * (1 - z[i,q,j])) - (M * (1 - z[k,q,l])))

In [60]:
# Equação 4 / (4-8)
for j in range(n):
  for l in range(n):
    for i in range(o-1):
      for k in range(o):
        for q in range(m):
          model.addConstr(y[k,l] >= y[i+1,j] - (M * (1 - x[q,i,j,k,l])) - (M * (1 - z[i,q,j])) - (M * (1 - z[k,q,l])))

In [61]:
# Equação 5 / (4-4)
for j in range(n):
  model.addConstr(gp.quicksum(z[0,q,j] for q in range(m)) <= 1)

In [62]:
# Equação 6 / (4-10)
for j in range(n):
  for l in range(n):
    if (j != l):
      for i in range(o-1): 
        for k in range(o):
          if (k > 0):
            for q in range(m):
              model.addConstr(x[q,i,j,k,l] + gp.quicksum(x[s,k-1,l,i+1,j] for s in range(m)) <= 1)

In [63]:
# Equação 7 / (4-9)
for j in range(n):
  for l in range(n):
    if (j != l):
      for i in range(o): 
        for k in range(o):
          for q in range(m):
            model.addConstr(x[q,i,j,k,l] + x[q,k,l,i,j] >= 1 - (M * (1 - z[i,q,j])) - (M * (1 - z[k,q,l])))

In [64]:
# Equação 8 / (4-3)
for j in range(n):
  for i in range(o):
    if (i > 0):
      model.addConstr(gp.quicksum(z[i,q,j] for q in range(m)) == 1 - gp.quicksum(f[k,j] for k in range(i-1)))

In [65]:
# Equação 9 / (4-7)
for j in TRAIN_OUT:
  for i in range(o):
    model.addConstr(f[i,j] <= z[i,PSA[j],j])

In [66]:
# Equação 10 / (4-5)
for j in TRAIN_OUT:
  model.addConstr(gp.quicksum(f[i,j] for i in range(o)) == 1)

In [67]:
# Equação 11 / (4-6)
for j in TRAIN_IN:
  model.addConstr(gp.quicksum(f[i,j] for i in range(o)) == 0)

In [68]:
# Equação 12 / (4-12)
for j in range(n):
  model.addConstr(z[0,PCH[j],j] == 1)

In [69]:
# Equação 13 / (4-13)
for j in TRAIN_OUT:
  model.addConstr(y[0,j] == RELEASE[j])

In [70]:
# Equação 14 / (4-14)
for j in TRAIN_OUT:
  model.addConstr(y[0,j] >= RELEASE[j])

In [71]:
# Equação 15 / (4-15)
for j in TRAIN_IN:
  for l in TRAIN_IN:
    model.addConstr(y[0,j] == y[0,l])

In [72]:
# Equação 16 / (4-11)
for j in range(n):
  for i in range(o-1):
    for q in range(m):
      model.addConstr(z[i+1,q,j] <= gp.quicksum(AMV[s][q] * z[i,s,j] for s in range(m)))

In [73]:
# Equação 17 / (4-16)
for j in TRAIN_OUT:
  for l in TRAIN_OUT:
    for i in range(o):
      model.addConstr(c[j] >= y[i,l])

In [74]:
# Equação 18 / (4-17)
for j in TRAIN_IN:
  model.addConstr(c[j] >= y[o-1,j])

In [75]:
# Equação 19 / (4-18)
for j in range(n):
  for i in range(o-1):
    model.addConstr(y[i+1,j] >= c[j] - M * (1 - gp.quicksum(f[k,j] for k in range(i))))

In [76]:
# Equação 20 / (4-19)
for j in TRAIN_OUT:
  model.addConstr(c_max >= c[j])

## Otimização do Problema

In [77]:
# Execute model
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15494 rows, 5643 columns and 78350 nonzeros
Model fingerprint: 0xc9975031
Variable types: 33 continuous, 5610 integer (5610 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+03]
Presolve removed 5426 rows and 1206 columns
Presolve time: 0.18s
Presolved: 10068 rows, 4437 columns, 48331 nonzeros
Variable types: 30 continuous, 4407 integer (4407 binary)

Root relaxation: objective 2.500000e+01, 170 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                      25.0000000   25.00000  0.00%     -    0s
     0     0   25.00000    0   23   25.00000   25.0000

## Resultados Encontrados

In [78]:
for var in model.getVars():
    if (var.X > 0):
        print(f"{var.VarName} = {var.X}")

y[0,1] = 10.0
y[1,0] = 5.0
y[1,1] = 15.0
y[2,0] = 10.0
y[2,1] = 20.0
y[3,0] = 15.0
y[3,1] = 25.0
y[4,0] = 25.0
y[4,1] = 25.0
y[5,0] = 30.0
y[5,1] = 25.0
y[6,0] = 35.0
y[6,1] = 25.0
y[7,0] = 40.0
y[7,1] = 25.0
y[8,0] = 45.0
y[8,1] = 25.0
y[9,0] = 50.0
y[9,1] = 25.0
y[10,0] = 55.0
y[10,1] = 25.0
y[11,0] = 60.0
y[11,1] = 25.0
y[12,0] = 65.0
y[12,1] = 25.0
y[13,0] = 70.0
y[13,1] = 25.0
y[14,0] = 75.0
y[14,1] = 25.0
z[0,1,1] = 1.0
z[0,2,0] = 1.0
z[1,1,0] = 1.0
z[1,2,1] = 1.0
z[2,0,0] = 1.0
z[2,3,1] = 1.0
z[3,0,0] = 1.0
z[4,0,0] = 1.0
z[5,1,0] = 1.0
z[6,4,0] = 1.0
z[7,4,0] = 1.0
z[8,4,0] = 1.0
z[9,5,0] = 1.0
z[10,5,0] = 1.0
z[11,5,0] = 1.0
z[12,5,0] = 1.0
z[13,5,0] = 1.0
z[14,5,0] = 1.0
x[0,0,0,2,0] = 1.0
x[0,0,0,13,1] = 1.0
x[0,0,0,14,1] = 1.0
x[0,0,1,0,0] = 1.0
x[0,0,1,2,1] = 1.0
x[0,0,1,3,0] = 1.0
x[0,0,1,3,1] = 1.0
x[0,0,1,4,0] = 1.0
x[0,0,1,4,1] = 1.0
x[0,0,1,5,1] = 1.0
x[0,0,1,6,0] = 1.0
x[0,0,1,6,1] = 1.0
x[0,0,1,7,0] = 1.0
x[0,0,1,7,1] = 1.0
x[0,0,1,8,0] = 1.0
x[0,0,1,8,1] = 1.0
x[0,